In [17]:
import datetime
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import seaborn as sns

#### Fonctions utiles

In [3]:
sondage = pd.read_excel('./../../34_Rapports/Observateur - Données qualitatives 2022/Nettoyage des données/test.xlsx')

In [86]:
park_occup = pd.read_csv('Nombre de transactions_[2017].csv')

In [87]:
nb_pers_veh = sondage.groupby(['Période'])['Nombre de personnes dans le véhicule'].mean().to_frame()

In [92]:
park_occup['datetime'] = list(map(datetime.datetime, park_occup.year, park_occup.month, park_occup.day))
park_occup['Période'] = ['Fin de semaine' if x.day_of_week in [5,6] else 'Semaine' for x in park_occup.datetime]

nb_veh = park_occup.groupby(['datetime', 'Période']).transactions_count_corrected.sum().groupby('Période').agg(['mean', 'size'])
nb_veh = nb_veh.rename(columns={'mean': 'Nombre de véhicules moyen', 'size':'Nombre de journée'})

In [93]:
data = nb_veh.join(nb_pers_veh)
data.loc['Tous', :] = [
    (
        (data.loc['Fin de semaine', 'Nombre de véhicules moyen'] * data.loc['Fin de semaine', 'Nombre de journée']) +
        (data.loc['Semaine', 'Nombre de véhicules moyen'] * data.loc['Semaine', 'Nombre de journée'])
     ) / (data.loc['Fin de semaine', 'Nombre de journée'] + data.loc['Semaine', 'Nombre de journée']),
    data.loc['Fin de semaine', 'Nombre de journée'] + data.loc['Semaine', 'Nombre de journée'],
    sondage['Nombre de personnes dans le véhicule'].mean()
]
data['Nombre de personnes en voiture'] = data['Nombre de véhicules moyen'] * data['Nombre de personnes dans le véhicule']
data = data.round(2)

In [99]:
data.to_excel('../Nombre_de_véhicules_par_jours.xlsx', index=True)